## Prepare Dataset

In [ ]:
import pandas as pd
from google.colab import files
import io

In [ ]:
uploaded = files.upload()

Saving Herbal - Dataset + Nama Herbal.csv to Herbal - Dataset + Nama Herbal (2).csv


## (Opsi lain) Ambil dataset dari gdrive

In [ ]:
!pip install gdown

In [ ]:
import gdown

# Ganti 'file_id' dengan ID file Google Drive
file_id = '1SZNXY6D1pMnTdSDGJ4NhTuMRFghjjT51'
url = f'https://drive.google.com/uc?id={file_id}'

# Ganti 'output' dengan nama file yang ingin Anda berikan
output = 'Herbal Dataset.csv'

gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1SZNXY6D1pMnTdSDGJ4NhTuMRFghjjT51
To: /content/Herbal Dataset.csv
100%|██████████| 312k/312k [00:00<00:00, 63.4MB/s]


'Herbal Dataset.csv'

## MODEL FIX

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Load and preprocess dataset
df = pd.read_csv("Herbal Dataset.csv")
nama_herbal_column_name = 'Nama Herbal'
khasiat_column_name = 'Khasiat'
saran_penyajian_column_name = 'Saran Penyajian'

# Save the original case of the columns
df['original_case_nama_herbal'] = df[nama_herbal_column_name]
df['original_case_khasiat'] = df[khasiat_column_name]
df['original_case_saran'] = df[saran_penyajian_column_name]

# Convert all words to lowercase
df[nama_herbal_column_name] = df[nama_herbal_column_name].str.lower()
df[khasiat_column_name] = df[khasiat_column_name].str.lower()
df[saran_penyajian_column_name] = df[saran_penyajian_column_name].str.lower()

# Menggabungkan kolom 'Khasiat' dan 'Saran Penyajian' menjadi satu teks
df['combined_text'] = df[nama_herbal_column_name] + ' ' + df[khasiat_column_name] + ' ' + df[saran_penyajian_column_name]

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=2, sublinear_tf=True)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_text'])

# Function to recommend rempah based on user input
def recommend_rempah(input_text):
    input_text = input_text.lower()
    input_vector = tfidf_vectorizer.transform([input_text])

    cosine_similarities = linear_kernel(input_vector, tfidf_matrix).flatten()

    # Filter out recommendations with cosine similarity = 0.0
    nonzero_indices = [i for i, sim in enumerate(cosine_similarities) if sim != 0.0]
    if not nonzero_indices:
        return None

    # Sort and get top 10 recommendations
    related_rempah_indices = sorted(nonzero_indices, key=lambda i: cosine_similarities[i], reverse=True)[:7]

    print("7 nilai cosine similarity tertinggi:\n")
    for index in related_rempah_indices:
        print(cosine_similarities[index])

    recommended_rempah = df.iloc[related_rempah_indices][['original_case_nama_herbal', 'original_case_khasiat', 'original_case_saran']]
    return recommended_rempah, cosine_similarities  # Kembalikan cosine_similarities bersama dengan rekomendasi rempah

# User input through Colab GUI
input_penyakit = input("Masukkan penyakit: ")
result = recommend_rempah(input_penyakit)

if result is not None:
    recommendations, cosine_similarities = result
    print("\n" + "="*500 + "\n")
    # Outputkan hasil per baris
    for index, row in recommendations.iterrows():
        print(f"Nama Herbal: {row['original_case_nama_herbal']}")
        print(f"Khasiat: {row['original_case_khasiat']}")
        print(f"Saran Penyajian: {row['original_case_saran']}")

        # Cetak nilai cosine similarity untuk setiap rekomendasi rempah
        cosine_similarity = cosine_similarities[index]
        print(f"Nilai Cosine Similarity: {cosine_similarity}")

        print("\n" + "="*500 + "\n")  # Pembatas antar rekomendasi
else:
    print(f'Maaf, tidak ada rekomendasi yang sesuai untuk penyakit {input_penyakit}.')


Masukkan penyakit: Diabetes Melitus
7 nilai cosine similarity tertinggi:

0.30733538980878855
0.17164990066534824
0.16138775638364805
0.15123207767202307
0.1507918331561274
0.14842801011185358
0.13889970612041752


Nama Herbal: Tapak dara
Khasiat: Daun tapak dara mengandung senyawa alkaloid, glikosida, dan flavonoid yang memiliki efek hipoglikemik, yaitu dapat menurunkan kadar gula darah. Hal ini dapat membantu mengobati diabetes melitus, terutama diabetes melitus tipe 2.
Saran Penyajian: 10-16 lembar daun tapak dara direbus dengan 3 gelas air sampai mendidih hingga tinggal 1 gelas. setelah dingin diminum, diulangi sampai sembuh.
Nilai Cosine Similarity: 0.30733538980878855


Nama Herbal: Kedelai
Khasiat: Kedelai (Glycine max) adalah tanaman herbal yang memiliki berbagai khasiat, salah satunya untuk membantu mengelola diabetes melitus. Kedelai mengandung senyawa-senyawa yang dapat meningkatkan sensitivitas insulin dan mengurangi kadar gula darah.
Saran Penyajian: 1 genggam biji kedelai